In [ ]:
# Write a query that returns the rate_type, loan_id, loan balance , 
# and a column that shows with what percentage the loan's balance 
# contributes to the total balance among the loans of the same rate type.

df = submissions
rate_type_total = df.groupby('rate_type')['balance'].sum().to_frame('balance_total').reset_index()
df = pd.merge(df, rate_type_total, on='rate_type')
df['contribution%'] = df.apply(lambda x: (x['balance'] / x['balance_total']) * 100, axis=1)
df = df[['id', 'rate_type', 'loan_id', 'balance', 'contribution%']]

In [ ]:
# Find the fraction (percentage divided by 100) of rides each weather-hour combination constitutes among all weather-hour combinations.
# Output the weather, hour along with the corresponding fraction.

outdf = lyft_rides.groupby(['weather', 'hour'], as_index=False)['gasoline_cost'].count().rename(columns={'gasoline_cost': 'count'})
outdf['probability'] = outdf['count']/ len(lyft_rides)
outdf = outdf[['weather', 'hour', 'probability']].sort_values(['weather', 'hour'])
outdf

In [ ]:
# Output ids of students with a median score from the writing SAT.
df = sat_scores[sat_scores['sat_writing'] == sat_scores['sat_writing'].median()]['student_id']
df

In [ ]:
# Find how many different origin airports exist?

us_flights['origin'].nunique()

In [ ]:
#Find all emails with duplicates.

employee = employee[['email', 'id']]
employee = employee.groupby('email', as_index=False)['id'].count().rename(columns={'id':'count'})
employee[employee['count']>1]

In [ ]:
# Find the average number of days between the booking and check-in dates for AirBnB hosts. 
# Order the results based on the average number of days in descending order.
# avg_days_between_booking_and_checkin DESC

df = airbnb_contacts
df['days_between_booking_and_checkin'] = (df['ds_checkin'].dt.date - df['ts_booking_at'].dt.date).dt.days
df = df.groupby('id_host')['days_between_booking_and_checkin'].mean().to_frame('avg_days_between_booking_and_checkin').reset_index()
df = df[['id_host', 'avg_days_between_booking_and_checkin']].sort_values('avg_days_between_booking_and_checkin', ascending=False).dropna()
df

In [ ]:
# Find all the users who were active for 3 consecutive days or more.


# Assuming df contains the DataFrame sf_events
df = sf_events
# Create an empty dictionary to store user activity
user_dates = {}

# Populate the dictionary with user activity
for i in range(len(df)):
    user_id = df.loc[i, 'user_id']
    date = df.loc[i, 'date']
    
    if user_id not in user_dates:
        user_dates[user_id] = []
    
    user_dates[user_id].append(date)

# Filter users who were active for 3 consecutive days or more
active_users = {}
for user_id, dates in user_dates.items():
    consecutive_days_count = 1
    for i in range(len(dates) - 1):
        if (dates[i + 1] - dates[i]).days == 1:
            consecutive_days_count += 1
            if consecutive_days_count >= 3:
                active_users[user_id] = dates[:3]  # Store the first 3 consecutive days
                break
        else:
            consecutive_days_count = 1

# Display the active users and their first 3 consecutive days
print(pd.DataFrame(active_users.keys()))


In [ ]:
# Find the top 5 highest paid and top 5 least paid employees in 2012.
# Output the employee name along with the corresponding total pay with benefits.
# Sort records based on the total payment with benefits in ascending order.

df = sf_public_salaries
df = df[df['year'] == 2012]
df = df[['id', 'totalpaybenefits']]
#df = pd.merge(df.sort_values('totalpaybenefits', ascending=False)[:5], df.sort_values('totalpaybenefits')[:5])
df_highest = df.sort_values('totalpaybenefits', ascending=False)[:5]
df_lowest = df.sort_values('totalpaybenefits', ascending=False)[-5:]
df_output = pd.concat([df_highest, df_lowest]).sort_values('totalpaybenefits')

In [ ]:
# Rank each host based on the number of beds they have listed. 
# The host with the most beds should be ranked 1 and the host with the least number of beds should be ranked last. 
# Hosts that have the same number of beds should have the same rank but there should be no gaps between ranking values. 
# A host can also own multiple properties.
# Output the host ID, number of beds, and rank from highest rank to lowest.

df = airbnb_apartments
df = df[['host_id', 'n_beds']]
df= df.groupby('host_id',as_index=False)['n_beds'].sum()
df = df.sort_values('n_beds', ascending=False)
df['rank'] = df['n_beds'].rank(method='dense', ascending=False)
df

In [ ]:
# Find the number of unique transactions and total sales for each of the product categories in 2017. 
# Output the product categories, number of transactions, and total sales in descending order. 
# The sales column represents the total cost the customer paid for the product so no additional calculations need to be done on the column.
# Only include product categories that have products sold.

df = pd.merge(wfm_transactions, wfm_products, on='product_id', how='inner')
df = df[df['transaction_date'].dt.year == 2017]
df = df.groupby('product_category', as_index=False).agg({'transaction_id': ['nunique'], 'sales': ['sum']})
df.columns = df.columns.droplevel(1)
df = df.sort_values('sales', ascending=False)
df

In [ ]:
# Write a query that returns a table containing the number of signups for each weekday and for each billing cycle frequency. 
# The day of the week standard we expect is from Sunday as 0 to Saturday as 6.
# Output the weekday number (e.g., 1, 2, 3) as rows in your table and the billing cycle frequency (e.g., annual, monthly, quarterly) as columns. 
# If there are NULLs in the output replace them with zeroes.

df = pd.merge(signups, plans, left_on='plan_id', right_on='id', how='inner')
df['weekday'] = df['signup_start_date'].dt.weekday
df = df.groupby(['weekday', 'billing_cycle'], as_index=False)['signup_id'].count()
df = pd.pivot_table(data = df, columns='billing_cycle', index='weekday', values='signup_id').reset_index().fillna(0)
df

In [ ]:
# Find the most profitable location. Write a query that calculates the average signup duration 
# and average transaction amount for each location, and then compare these two measures together 
# by taking the ratio of the average transaction amount and average duration for each location.
# Your output should include the location, average duration, average transaction amount, and ratio. 
# Sort your results from highest ratio to lowest.

df = pd.merge(signups, transactions, on='signup_id', how='inner')
df['signup_duration'] = (df['signup_stop_date'] - df['signup_start_date']).dt.days
df = df.groupby('location', as_index=False).agg({'amt': 'mean', 'signup_duration': 'mean'})
df['ratio'] = df['amt'] / df['signup_duration']
df = df.sort_values('ratio', ascending=False)
df

In [ ]:
# Find the monthly retention rate of users for each account separately for Dec 2020 and Jan 2021. 
# Retention rate is the percentage of active users an account retains over a given period of time. 
# In this case, assume the user is retained if he/she stays with the app in any future months. 
# For example, if a user was active in Dec 2020 and has activity in any future month, consider them retained for Dec. 
# You can assume all accounts are present in Dec 2020 and Jan 2021. 
# Your output should have the account ID and the Jan 2021 retention rate divided by Dec 2020 retention rate.

# Get Accounts active in December
dec2020 = sf_events[(sf_events['date'].dt.year == 2020) & (sf_events
['date'].dt.month == 12)].drop_duplicates(subset = ['account_id', 'user_id'])
dec2020['in_dec'] = 1
 
# Get accounts active after December
aft_dec2020 = sf_events[sf_events['date'].dt.date > pd.to_datetime
('2020-12-31')].drop_duplicates(subset = ['account_id', 'user_id'])
aft_dec2020['aft_dec'] = 1
 
# Merge the datasets to get December retention numbers
dec_merged = pd.merge(dec2020, aft_dec2020, on = 
['account_id', 'user_id'], how = 'left')
dec_summ = dec_merged.groupby(by = ['account_id'], as_index = False).sum()
dec_summ['dec_retention'] = dec_summ['aft_dec'] / dec_summ['in_dec']

# Repeat the process for Jan 2021
jan2021 = sf_events[(sf_events['date'].dt.year == 2021) & (sf_events
['date'].dt.month == 1)].drop_duplicates(subset = ['account_id', 'user_id'])
jan2021['in_jan'] = 1
aft_jan2021 = sf_events[sf_events['date'].dt.date > pd.to_datetime
('2021-01-31')].drop_duplicates(subset = ['account_id', 'user_id'])
aft_jan2021['aft_jan'] = 1
 
jan_merged = pd.merge(jan2021, aft_jan2021, on = 
['account_id', 'user_id'], how = 'left')
jan_summ = jan_merged.groupby(by = ['account_id'], as_index = False).sum()
jan_summ['jan_retention'] = jan_summ['aft_jan'] / jan_summ['in_jan']
 
# Merge the Dec20 and Jan21 datasets, calculate the relative retention rate 
# and output.
final_merge = pd.merge(dec_summ, jan_summ, on = 'account_id', how = 'left')
final_merge['retention'] = final_merge['jan_retention'] / final_merge['dec_retention']
final_merge[['account_id', 'retention']]

In [ ]:
# ABC Corp is a mid-sized insurer in the US and in the recent past their fraudulent claims have increased significantly for their personal auto insurance portfolio. 
# They have developed a ML based predictive model to identify propensity of fraudulent claims. 
# Now, they assign highly experienced claim adjusters for top 5 percentile of claims identified by the model.
# Your objective is to identify the top 5 percentile of claims from each state. 
# Your output should be policy number, state, claim cost, and fraud score.

fraud_score['percentile'] = fraud_score.groupby('state', as_index=False)['fraud_score'].rank(pct=True)
fraud_score = fraud_score[fraud_score['percentile']>0.95]
fraud_score

In [ ]:
# For each service, calculate the percentage of incomplete orders along with the revenue loss percentage. 
# Your output should include the name of the service, percentage of incomplete orders, and revenue loss from the incomplete orders.

completed = uber_orders[uber_orders['status_of_order'] == 'Completed']
incomplete = uber_orders[uber_orders['status_of_order'] != 'Completed']
complete = completed.groupby('service_name', as_index=False).agg({'number_of_orders': 'sum', 'monetary_value': 'sum'})
incomplete = incomplete.groupby('service_name', as_index=False).agg({'number_of_orders': 'sum', 'monetary_value': 'sum'})
merged = pd.merge(complete, incomplete, on='service_name')
merged['percent_of_incomplete'] = (merged['number_of_orders_y'] / merged['number_of_orders_x']) * 100
merged['rev_loss'] = (merged['monetary_value_y'] / (merged['monetary_value_y'] + merged['monetary_value_x'])) * 100

merged = merged.drop(columns=['number_of_orders_x', 'monetary_value_x', 'number_of_orders_y',	'monetary_value_y'])
merged

In [ ]:
# Some forecasting methods are extremely simple and surprisingly effective. 
# Naïve forecast is one of them; we simply set all forecasts to be the value of the last observation. 
# Our goal is to develop a naïve forecast for a new metric called "distance per dollar" defined as the (distance_to_travel/monetary_cost) in our dataset and measure its accuracy.
# To develop this forecast,  sum "distance to travel"  and "monetary cost" values at a monthly level before calculating "distance per dollar". 
# This value becomes your actual value for the current month. 
# The next step is to populate the forecasted value for each month. 
# This can be achieved simply by getting the previous month's value in a separate column. 
# Now, we have actual and forecasted values. This is your naïve forecast. 
# Let’s evaluate our model by calculating an error matrix called root mean squared error (RMSE). 
# RMSE is defined as sqrt(mean(square(actual - forecast)). 
# Report out the RMSE rounded to the 2nd decimal spot.

df = uber_request_logs
df = df[['request_date', 'distance_to_travel', 'monetary_cost']]
df['month'] = df['request_date'].dt.month
df = df.groupby('month', as_index=False).agg({'distance_to_travel': 'sum', 'monetary_cost':'sum'})
df['distance_per_dollar'] = df['distance_to_travel'] / df['monetary_cost']
df['forecast'] = df['distance_per_dollar'].shift(1).fillna(0)

rmse = round((((df['distance_per_dollar'] - df['forecast'])**2).mean())**0.5, 2)
rmse

In [ ]:
# Find the top business categories based on the total number of reviews. 
# Output the category along with the total number of reviews. 
# Order by total reviews in descending order.

df = yelp_business
df['categories'] = df['categories'].apply(lambda x: x.split(';'))
df = df[['categories', 'review_count']]
df = df.explode('categories').groupby('categories', as_index=False).sum().sort_values('review_count', ascending=False)
df

In [ ]:
# Find the number of times each word appears in drafts.
# Output the word along with the corresponding number of occurrences.

import pandas as pd
import string

def remove_punctuation(x):
    return ''.join(char for char in x if char not in string.punctuation)
    
df = google_file_store
df = df[df['filename'].str.contains('draft')]
df['contents'] = df['contents'].apply(remove_punctuation).apply(lambda x: x.split(" "))
df = df.explode('contents')
df['count'] = [1]*len(df)
df = df.groupby('contents', as_index=False)['count'].sum()
df

#OR

import pandas as pd
import numpy as np

draft = google_file_store[google_file_store['filename'].str.contains('draft')]
result = draft.contents.str.split('\W+', expand=True).stack().value_counts().reset_index()

In [ ]:
# The company you work for has asked you to look into the average order value per hour 
# during rush hours in the San Jose area. 
# Rush hour is from 15H - 17H inclusive.
# You have also been told that the column order_total represents the gross order total for each order. 
# Therefore, you'll need to calculate the net order total.
# The gross order total is the total of the order before adding the tip and deducting the discount and refund.
# Use the column customer_placed_order_datetime for your calculations.

df = delivery_details[delivery_details['delivery_region'] == 'San Jose']
df = df[['customer_placed_order_datetime', 'order_total', 'discount_amount', 'tip_amount',	 'refunded_amount']]
df['net_total'] = df['order_total'] + df['tip_amount'] - df['discount_amount'] - df['refunded_amount']
df = df.drop(['order_total', 'discount_amount', 'tip_amount',	 'refunded_amount'], axis=1)
df['peak_hours'] = df['customer_placed_order_datetime'].dt.hour
df = df[(df['peak_hours'] >=15) &  (df['peak_hours'] <=17)]
df = df.groupby('peak_hours')['net_total'].mean()
df

In [ ]:
# Find the email activity rank for each user. Email activity rank is defined by the total number of emails sent. 
# The user with the highest number of emails sent will have a rank of 1, and so on. 
# Output the user, total emails, and their activity rank. 
# Order records by the total emails in descending order. 
# Sort users with the same number of emails in alphabetical order.
# In your rankings, return a unique value (i.e., a unique rank) even if multiple users have the same number of emails. 
# For tie breaker use alphabetical order of the user usernames.

df = google_gmail_emails
df['count'] = [1]*len(df)
df = df.groupby('from_user', as_index=False)['count'].sum().sort_values(['count', 'from_user'], ascending= [False, True])
df['rank'] = [i for i in range(1, len(df)+1)]
df

In [ ]:
# Write a query that returns a number of users who are exclusive to only one client. 
# Output the client_id and number of exclusive users.

df = fact_events[['user_id', 'client_id']]
num_users = df.groupby('user_id', as_index=False)['client_id'].nunique()
exclusive = num_users[num_users['client_id']==1]
exclusive = exclusive.drop(columns = 'client_id')
exclusive = pd.merge(exclusive, df, on='user_id')
exclusive = exclusive.groupby('client_id', as_index=False)['user_id'].nunique()
exclusive

In [ ]:
# Calculate the win-to-nomination ratio for each nominee. Output the ratio and the nominee's name. 
# Order the results based on the ratio in descending order to show nominees with the highest ratio on top.

df = oscar_nominees[['nominee', 'winner']]
df = pd.pivot_table(columns='winner', index='nominee', data=df, aggfunc=len, fill_value=0).reset_index()
df['ratio'] = df[True] / (df[True] + df[False])
df = df.sort_values('ratio', ascending=False).drop(columns=[False, True])
df

In [ ]:
# Write a query that returns an array of signup ids by day. Order by most signups in descending order.

df = signups[['signup_id', 'signup_start_date']]
df['signup_start_date'] = df['signup_start_date'].dt.date
df = df.groupby(by = ['signup_start_date'])['signup_id'].apply(list).reset_index()
df['num_signups'] = df['signup_id'].apply(len)
df = df.sort_values(['num_signups', 'signup_start_date'], ascending=[False, True]).drop(columns='num_signups')
df

In [ ]:
# You work for a multinational company that wants to calculate total sales across all their countries they do business in.
# You have 2 tables, one is a record of sales for all countries and currencies the company deals with, and the other holds currency exchange rate information.
# Calculate the total sales, per quarter, for the first 2 quarters in 2020, and report the sales in USD currency.

exng = sf_exchange_rate
sales = sf_sales_amount
sales = sales[sales['sales_date'].dt.year == 2020]
sales['quarter'] = sales['sales_date'].dt.quarter
sales = sales[(sales['quarter'] == 1) | (sales['quarter'] == 2)]
sales['sales_date'] = sales['sales_date'].dt.date
exng['date'] = exng['date'].dt.date
sales = pd.merge(sales, exng, left_on=['sales_date', 'currency'], right_on=['date', 'source_currency'], how="left").reset_index()
sales['amount'] = sales['sales_amount'] * sales['exchange_rate']
sales = sales[['quarter', 'amount']]
sales = sales.groupby('quarter', as_index=False)['amount'].sum()
sales

In [ ]:
# From users who had their first session as a viewer, how many streamer sessions have they had? 
# Return the user id and number of sessions in descending order. 
# In case there are users with the same number of sessions, order them by ascending user id.

df = twitch_sessions
df = df.sort_values(['user_id', 'session_start'], ascending=[True, True])
df1 = df.groupby('user_id')['session_type'].apply(list).reset_index()
users = []
for i in range(len(df1)):
    if df1.iloc[i,1][0] == 'viewer':
        users.append(df1.iloc[i,0])
        
df = df[df['user_id'].isin(users)]
df = df.groupby('user_id', as_index=False)['session_id'].count()
df['session_id'] = df['session_id'] - 1
df = df.sort_values('user_id', ascending=True)
df

In [ ]:
# You’re given a table of Uber rides that contains the mileage and the purpose for the business expense.  
# You’re asked to find business purposes that generate the most miles driven for passengers that use Uber for their business transportation. 
# Find the top 3 business purpose categories by total mileage.

df = my_uber_drives
df = df[df['category'] =='Business']
df = df.dropna(subset=['purpose'])
df = df.groupby('purpose', as_index=False)['miles'].sum()
df = df.sort_values('miles', ascending=False)[:3]
df